# DVA263 project

# Libraries

Declaring all libraries in use

In [34]:
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L2
import os
import cnn as cnn
import preprocessing as pre
import visualization as vis
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

from Project.Code.preprocessing import convert_to_numpy



# Load Dataset

In [35]:
def load_cnn():
    """Load the datasets for the CNN input"""
    CNN_train = "../dataset/CNN_Input/train/"
    CNN_test = "../dataset/CNN_Input/test/"
    CNN_train = pre.import_images(CNN_train, (150,150),batch_size=16, dataset=True)
    CNN_test = pre.import_images(CNN_test, (150,150), batch_size=16,  dataset=True)
    return CNN_train, CNN_test

In [36]:
def load(image_size):
    train_path = "../dataset/Project/train/"
    test_path = "../dataset/Project/test/"

    CNN_train = pre.import_images(train_path, image_size, dataset=True)
    ALL_test = pre.import_images(test_path, image_size, dataset=True)
    
    print(ALL_test)
    y_test, X_test = convert_to_numpy(ALL_test)
    return y_test, X_test

In [37]:
image_size = (152, 152)

y_test, X_test = load(image_size)

print(y_test.shape)
print(X_test.shape)

#for label in y_test: 
    #print(label)

Found 259 files belonging to 4 classes.
Found 33 files belonging to 4 classes.
<_PrefetchDataset element_spec=(TensorSpec(shape=(152, 152, 3), dtype=tf.float32, name=None), TensorSpec(shape=(4,), dtype=tf.float32, name=None))>
(132,)
(5016, 152, 3)


# Prepare Data

In [38]:
def prep_cnn(cnn_train, cnn_test):
    """Add noise images to train and test set and normalize"""
    g_noisy_train_data = cnn_train.map(pre.add_gaussian_noise)
    snp_noisy_train_data = cnn_train.map(pre.add_salt_and_pepper_noise)
    combined_train_data = cnn_train.concatenate(g_noisy_train_data).concatenate(snp_noisy_train_data)

    percentage_noisy_images = 0.2
    total_test_images = len(CNN_test) * 16  # Total images in test data
    num_noisy_images = int(total_test_images * percentage_noisy_images)

    g_noisy_test_data = cnn_test.take(num_noisy_images).map(pre.add_gaussian_noise)
    snp_noisy_test_data = cnn_test.take(num_noisy_images).map(pre.add_salt_and_pepper_noise)
    remaining_test_data = cnn_test.skip(num_noisy_images)

    combined_test_data = g_noisy_test_data.concatenate(remaining_test_data)
    combined_test_data = combined_test_data.concatenate(snp_noisy_test_data)

    test_data_norm, train_data_norm = pre.normalize_dataset(combined_train_data, combined_test_data)
    return train_data_norm, test_data_norm


# Build and Compile Model

In [39]:
def build_cnn():
    model = cnn.build_cnn_model()
    return model

#  Train Model

In [40]:
def train_cnn(model,cnn_train, cnn_test):
    history = cnn.train_model(model, cnn_train, cnn_test)
    return history

# Putting everything together

Main section putting each part together and running the classification task

In [41]:
CNN_train, CNN_test = load_cnn()
CNN_train, CNN_test = prep_cnn(CNN_train, CNN_test)

CNN_model = build_cnn()
CNN_history = train_cnn(CNN_model, CNN_train, CNN_test)
CNN_model.predict(CNN_test)


Found 83 files belonging to 4 classes.
Found 62 files belonging to 4 classes.
Epoch 1/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - accuracy: 0.2582 - loss: 1.4034 - val_accuracy: 0.2742 - val_loss: 1.3853 - learning_rate: 0.0010
Epoch 2/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.2373 - loss: 1.3864 - val_accuracy: 0.2258 - val_loss: 1.3853 - learning_rate: 0.0010
Epoch 3/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.3012 - loss: 1.3828 - val_accuracy: 0.2742 - val_loss: 1.3798 - learning_rate: 0.0010
Epoch 4/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.2761 - loss: 1.3761 - val_accuracy: 0.4032 - val_loss: 1.3776 - learning_rate: 0.0010
Epoch 5/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.4479 - loss: 1.3726 - val_accuracy: 0.3306 - val_loss: 1.3554 - learning_rate: 0.0010
Epoch 6/70
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.4010 - loss: 1.3406 - val_accuracy: 0.3226 - val_loss: 1.3347 - learning_rate: 0.0010
Epoch 7/70
18/18

array([[9.44427215e-04, 4.63477150e-03, 3.07910936e-03, 9.91341650e-01],
       [4.22051846e-04, 9.99385595e-01, 1.87288088e-10, 1.92299776e-04],
       [3.17353370e-05, 2.21923801e-06, 9.99158382e-01, 8.07775243e-04],
       [1.54788583e-03, 9.98430550e-01, 6.31627063e-06, 1.52828634e-05],
       [3.74128774e-07, 2.77674360e-07, 9.77351487e-01, 2.26478633e-02],
       [9.99741018e-01, 2.53549457e-04, 5.39943176e-06, 3.46967397e-08],
       [1.23234747e-06, 9.99988079e-01, 8.66503758e-10, 1.07452552e-05],
       [9.98825371e-01, 1.17161474e-03, 2.65257540e-06, 4.29283403e-07],
       [4.51234449e-03, 9.94784772e-01, 2.12390262e-07, 7.02685036e-04],
       [4.11388419e-06, 4.25321935e-03, 2.18531757e-04, 9.95524168e-01],
       [6.90035307e-10, 1.68413354e-08, 9.99818623e-01, 1.81431795e-04],
       [9.99995112e-01, 4.92226036e-06, 5.74355785e-10, 3.15094750e-09],
       [9.98329461e-01, 1.67028781e-03, 2.84933549e-07, 4.87245408e-08],
       [9.81687263e-05, 4.44655188e-06, 9.97689843e